In [5]:
import requests
import pandas as pd
import numpy as np
import json
import seaborn as sb
import urllib

# We download and store the data from the website of the city of New York

In [5]:
page_one = requests.get('https://data.cityofnewyork.us/resource/9w7m-hzhe.json?$limit=50000').json()
df = pd.DataFrame(page_one)

for i in range(1,8):
    next_page = requests.get('https://data.cityofnewyork.us/resource/9w7m-hzhe.json?$limit=50000&$offset=' + str(int(i*5))+ '0000').json()
    df2 = pd.DataFrame(next_page)
    df = df.append(df2)

df.drop_duplicates()
df.to_csv("raw_data.csv",index=False)

,action,boro,building,camis,critical_flag,cuisine_description,dba,grade,grade_date,inspection_date,inspection_type,phone,record_date,score,street,violation_code,violation_description,zipcode
0,Violations were cited in the following area(s).,QUEENS,13204,50039146,Not Critical,Caribbean,BROWN BETTY RESTAURANT,A,2016-09-02T00:00:00.000,2016-09-02T00:00:00.000,Cycle Inspection / Initial Inspection,7183236438,2018-11-03T06:01:38.000,4,LIBERTY AVE,09C,Food contact surface not properly maintained.,11419
1,Violations were cited in the following area(s).,BROOKLYN,8814,50072006,Critical,Pizza/Italian,PATSY'S PIZZERIA OF BAY RIDGE,NaN,NaN,2018-01-11T00:00:00.000,Pre-permit (Operational) / Initial Inspection,7187453003,2018-11-03T06:01:38.000,34,3RD AVE,02G,Cold food item held above 41Âº F (smoked fish ...,11209
2,Violations were cited in the following area(s).,MANHATTAN,10,50012118,Critical,Japanese,NORIKOH,A,2015-10-13T00:00:00.000,2015-10-13T00:00:00.000,Cycle Inspection / Re-inspection,6469186778,2018-11-03T06:01:38.000,10,E 39TH ST,06A,Personal cleanliness inadequate. Outer garment...,10016
3,Violations were cited in the following area(s).,QUEENS,12432A,50070589,Not Critical,Thai,OCHA THAI CUISINE,NaN,NaN,2018-04-18T00:00:00.000,Cycle Inspection / Initial Inspection,7182610008,2018-11-03T06:01:38.000,7,QUEENS BLVD,10F,Non-food contact surface improperly constructe...,11415
4,Violations were cited in the following area(s).,BROOKLYN,592,50071826,Critical,Soul Food,BLACK NILE SEAFOOD & SOUL FOOD,A,2018-10-18T00:00:00.000,2018-10-18T00:00:00.000,Cycle Inspection / Initial Inspection,3478798911,2018-11-03T06:01:38.000,12,NOSTRAND AVE,04N,Filth flies or food/refuse/sewage-associated (...,11216
5,Violations were cited in the following area(s).,MANHATTAN,2755,50004346,Critical,American,KFC,Z,2018-09-04T00:00:00.000,2018-09-04T00:00:00.000,Cycle Inspection / Re-inspection,2126659430,2018-11-03T06:01:38.000,52,BROADWAY,05D,Hand washing facility not provided in or near ...,10025
6,Violations were cited in the following area(s).,QUEENS,14932,50033122,Critical,Korean,PARTY WELL RESTAURANT & ORIENTAL BAKERY,B,2017-04-19T00:00:00.000,2017-04-19T00:00:00.000,Cycle Inspection / Re-inspection,7188860800,2018-11-03T06:01:38.000,27,41ST AVE,06E,"Sanitized equipment or utensil, including in-u...",11355
7,Violations were cited in the following area(s).,BRONX,3827,50060018,Critical,Caribbean,SEA JAMMINS CARIBBEAN CUISINE,NaN,NaN,2018-01-09T00:00:00.000,Cycle Inspection / Initial Inspection,7189943875,2018-11-03T06:01:38.000,17,BRONXWOOD AVE,04L,Evidence of mice or live mice present in facil...,10469
8,Violations were cited in the following area(s).,QUEENS,6910,41441583,Not Critical,Filipino,FRITZIES BAKE SHOP,A,2017-01-24T00:00:00.000,2017-01-24T00:00:00.000,Cycle Inspection / Re-inspection,7185070205,2018-11-03T06:01:38.000,11,ROOSEVELT AVENUE,10F,Non-food contact surface improperly constructe...,11377
9,Violations were cited in the following area(s).,BROOKLYN,1319,50000510,Not Critical,Jewish/Kosher,JERUSALEM STEAKHOUSE,NaN,NaN,2016-04-18T00:00:00.000,Cycle Inspection / Initial Inspection,7182588899,2018-11-03T06:01:38.000,23,AVENUE J,10F,Non-food contact surface improperly constructe...,11230


In [28]:
df = pd.read_csv("raw_data.csv",)
df_restaurant = df.drop_duplicates("camis")
df_restaurant = df_restaurant[["dba","street","zipcode"]]
df_restaurant["searched"] = False
df_restaurant.to_csv("restaurants_raw_data.csv",index=False)
print(df_restaurant)

                                             dba               street  \
0                         BROWN BETTY RESTAURANT          LIBERTY AVE   
1                  PATSY'S PIZZERIA OF BAY RIDGE              3RD AVE   
2                                        NORIKOH            E 39TH ST   
3                              OCHA THAI CUISINE          QUEENS BLVD   
4                 BLACK NILE SEAFOOD & SOUL FOOD         NOSTRAND AVE   
5                                            KFC             BROADWAY   
6        PARTY WELL RESTAURANT & ORIENTAL BAKERY             41ST AVE   
7                  SEA JAMMINS CARIBBEAN CUISINE        BRONXWOOD AVE   
8                             FRITZIES BAKE SHOP     ROOSEVELT AVENUE   
9                           JERUSALEM STEAKHOUSE             AVENUE J   
10                                MARIA PIZZERIA  FT HAMILTON PARKWAY   
11                      EL PENACHO II RESTAURANT      NEW UTRECHT AVE   
12                                     PIZZA BOY   

# We create a function that stores the basic info from the businesses from the Yelp API

In [6]:
YELP_API_KEY = ""
YELP_CLIENT_ID = ""
def yelp_search(term,location):
    headers = {"Authorization": "Bearer " + YELP_API_KEY}
    payload = {'term': term, 'location': location}
    params = urllib.parse.urlencode(payload) #encodes the data
    r = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params=params)
    results = r.json()
    try:    
        yelp_id = results["businesses"][0]["id"]
        yelp_name = results["businesses"][0]["name"]
        if "price" in results["businesses"][0].keys():
            yelp_price = results["businesses"][0]["price"]
        else:
            yelp_price = None
        yelp_rating = results["businesses"][0]["rating"]
        yelp_review_count = results["businesses"][0]["review_count"]
        yelp_transaction_type = results["businesses"][0]["transactions"]
        yelp_latitude = results["businesses"][0]["coordinates"]["latitude"]
        yelp_longitude = results["businesses"][0]["coordinates"]["longitude"]
        if yelp_transaction_type == []:
            yelp_transaction_type = "N/A"
    except:
        print(results)
        yelp_id = None
        yelp_price = None
        yelp_rating = None
        yelp_review_count = None
        yelp_transaction_type = None
        yelp_name = None
        yelp_latitude = None
        yelp_longitude = None
    return (yelp_id, yelp_price, yelp_rating, yelp_review_count, yelp_transaction_type,yelp_name,yelp_latitude,yelp_longitude)

# We create a function that fetches additional details from businesses using the Yelp API

In [7]:
def yelp_details(yelp_id):
    headers = {"Authorization": "Bearer " + YELP_API_KEY}
    r = requests.get('https://api.yelp.com/v3/businesses/'+yelp_id, headers=headers,)
    results = r.json()
    try:
        yelp_claimed = results["is_claimed"]
        try:
            yelp_hours = results["hours"]
        except: 
            print("Yelp hours unavailable!")
            yelp_hours = None
    except:
        print("Yelp details unavailable!")
        yelp_claimed = None
        yelp_hours = None
    return (yelp_claimed, yelp_hours)

# We leverage our functions to download and store the data from Yelp

In [14]:
df_restaurant = pd.read_csv("restaurants_raw_data.csv")
print(len(df_restaurant))
print(len(df_restaurant[df_restaurant["searched"] == True ]))
count =1
df_restaurant = df_restaurant.sort_values(by=["searched"],ascending = True)
for restau in df_restaurant.index[0:2000]:
    if df_restaurant.loc[restau,"searched"] == False:
        print(df_restaurant.loc[restau,"dba"])
        search_results = yelp_search(str(df_restaurant.loc[restau,"dba"]),str(df_restaurant.loc[restau,"street"]) + " NY " + str(df_restaurant.loc[restau,"zipcode"]))
        df_restaurant.loc[restau,"yelp_id"] = search_results[0]
        df_restaurant.loc[restau,"yelp_price"] = search_results[1]
        df_restaurant.loc[restau,"yelp_rating"] = search_results[2]
        df_restaurant.loc[restau,"yelp_review_count"] = search_results[3]
        if type(search_results[4]) == list:
            df_restaurant.loc[restau,"yelp_transaction_type"] = ';'.join(str(e) for e in search_results[4])
        df_restaurant.loc[restau,"yelp_latitude"] = search_results[6]
        df_restaurant.loc[restau,"yelp_longitude"] = search_results[7]
        df_restaurant.loc[restau,"yelp_name"] = search_results[5]
        if type(df_restaurant.loc[restau,"yelp_id"]) == str:
            details_results = yelp_details(df_restaurant.loc[restau,"yelp_id"])
        df_restaurant.loc[restau,"yelp_is_claimed"] = details_results[0]
        df_restaurant.loc[restau,"yelp_hours"] = details_results[1]
        df_restaurant.loc[restau,"searched"] = True
        print(count,search_results[5])
        count = count +1    
        df_restaurant.to_csv("restaurants_raw_data.csv",index=False)

26817
25042
ELMO
1 Elmo
OUTBACK STEAKHOUSE
2 Outback Steakhouse
TIKKA INDIAN GRILL
3 Tikka Indian Grill
PADDY REILLY'S BAR
4 Paddy Reilly's Music Bar
STARBUCKS
5 Starbucks
PEPPA POT JAMAICAN CUISINE
6 The Door
SNAFU 28
7 Snafu 28
OVENLY
8 Ovenly
PERSHING SQUARE CAFE
9 Pershing Square
NU HOTEL
10 NU Hotel
P.J. CLARKE'S
11 P.J. Clarke's
PIZZA HUT
12 Pizza Hut
PIZZA PETE'S
13 Pizza Pete's
PIZZA MIA
14 Pizza Mia
P.J. CLARKE'S
15 P.J. Clarke's
OUSIA
16 Ousia
OUTBACK STEAKHOUSE
17 Outback Steakhouse
OUTBACK STEAKHOUSE
18 Outback Steakhouse
LIN'S GOURMET
Yelp hours unavailable!
19 Lin's Gourmet
LIN'S LUCKY RESTAURANT
20 Joe's Shanghai
PAPILLES
21 Papilles
SUBWAY
Yelp hours unavailable!
22 Subway
NY GYRO XPRESS
23 NY Gyro Xpress
MAMA KITCHEN NY
24 Saraghina
MET LIFE TEA BAR / GRAB & GO
25 Lady M Cake Boutique
AWASH ETHIOPIAN RESTAURANT
26 Awash Ethiopian Restaurant
CAFE LATTE
27 Café Latté
ASO ROCK LOUNGE AND RESTAURANT
28 Aso Rock Lounge & Restaurant
BROOKLYN SOCIAL
29 Brooklyn Social
MAKANA


RAENU THAI
181 Raenu Continental Thai Restaurant
POKE NOM
182 Poke Nom
PIZZA FACTORY
183 Pizza Factory
PIZZA GARDEN
184 Pizza Garden
PEKING
185 Peking Kitchen
PERROS Y VAINAS
186 Perros y Vainas
PIES-N-THIGHS
187 Pies-n-Thighs
PLUG UGLIES
Yelp hours unavailable!
188 Plug Uglies
PEKING CHINESE RESTAURANT
Yelp hours unavailable!
189 Peking Chinese Restaurant
PICCOLO CAFE
190 Piccolo Cafe
PAKIZA RESTAURANT
Yelp hours unavailable!
191 Pakiza Restaurant
S & R LE JUICE BAR
192 S&R Le Juice Bar & Cafe
PINK MOOSE
193 Pink Moose
PEGGY DEMPSEYS
194 Peggy Dempseys
NASSAU BAR
195 Nassau Bar
CAFE UN DEUX TROIS
196 Cafe Un Deux Trois
MY PIE
197 My Pie
MY CUBAN SPOT
198 My Cuban Spot
MYUNG SAN RESTAURANT
199 Myung San
MULDOON'S IRISH PUB & RESTAURANT
200 Muldoon's
PHIL'S
201 Phil's
POCKET BAR NYC
202 Pocket Bar NYC
PACHANGA PATTERSON
203 Pachanga Patterson
NEW HONG KONG RESTAURANT
Yelp hours unavailable!
204 New Hong Kong House
CREME & SUGAR
205 Creme&sugar
PICK-A-BAGEL/DUMPLING ROOM
206 Barney Green

BROOKLYN BREWHOUSE
364 Brooklyn Brewhouse
2 BROS PIZZA
365 2 Bros Pizza
CIBO EXPRESS GOURMET MARKET
Yelp hours unavailable!
366 Cibo Express Gourmet Market
MINT PORCH CAFE
367 The River Café
ANGIE'S BREAKFAST SPOT
368 Angie's Breakfast Spot
BURGER KING
369 Burger King
DANNY BLAINE
370 Danny Blaine's Bay St Saloon
ANALOGUE
371 Analogue
HONG KONG KITCHEN
Yelp hours unavailable!
372 Hong Kong Kitchen
CRAFT EMPANADAS BY S&R
373 Red Rooster Harlem
TEA SHOP 168 & BAKERY
374 Tea Shop 168 & Bakery
ANNABELLE'S
375 Annabelle's Pastaria
KONDITORI
376 Konditori
nan
377 Nan Xiang Xiao Long Bao
MAMA RAO'S RESTAURANT & BAR
378 Mama Rao Cucina Italiana
NEW CITY ASIAN CUISINE
379 Amituofo Vegan Cuisine
MOON LIGHT  GRILL
380 786 Moon Light Grill
RADIO CITY MUSIC HALL
Yelp hours unavailable!
381 Radio City Music Hall
ELI ZABAR
382 Eli's Manhattan Market
nan
383 Nan Xiang Xiao Long Bao
nan
384 Nan Xiang Xiao Long Bao
ALMAZ RESTAURANT
385 Almaz Restaurant
MAGGIE MOO'S
386 Maggie Moo's Ice Cream & Treatery


PAPER RAIN
545 MRK Computer Services
PAPA JOHN'S
Yelp hours unavailable!
546 Papa John's
PAPA JOHN'S
Yelp hours unavailable!
547 Papa John's
ONE WAY DELI
Yelp hours unavailable!
548 One Way Deli
ONE AND ONE
549 ONE AND ONE
QUISQUEYA RESTAURANT
550 Quisqueya Restaurant
R & T PIZZA
551 Koronet Pizza
PATISSERIE
552 Takahachi Bakery
PAPA JOHN'S
Yelp hours unavailable!
553 Papa John's Pizza
Quintessence Restaurant
554 Quintessence
ONE MORE DRINKS
555 ABC Drink-One More
PAPAYA KING
556 Papaya King
PEPE GIALLO
557 Pepe Giallo
LCLE BAR
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -73.8226318359375, 'latitude': 40.760257831072536}}}
558 None
RAINBOW BAKERY AND CAFE
559 Martha’s Country Bakery
ONIS RESTAURANT
560 La Morada Restaurant
POKE BOWL
561 Pokéworks
LA ISLA RESTAURANT
Yelp hours unavailable!
562 La Isla Cuchifrito
PATA PA-PLEAN
563 Pata Paplean Bar
COCO'S PIZZA & RESTAURANT
564 La Bona Pizza & Pasta
PARK ITALIAN GOURMET
565 Park Italian Gourmet
OOTOYA
566 Ootoya Time

N.Y. TRACKS CAFE
717 Cafe Madeline
POPEYES
718 Popeyes Louisiana Kitchen
PERGOLA DES ARTISTES
719 Pergola Des Artistes
PIECE OF VELVET
720 Piece of Velvet
POLOSUD
721 Polosud Gelato Coffee Pastries
SNACK TIME GRILL
722 Sushi Time
LA GRINGA BAR & GRILL
723 La Gringa Bar and Grill
PINK LOTUS GOURMET
724 MáLà Project
DAVIDS TEA
725 DAVIDsTEA
AMC THEATRES
Yelp hours unavailable!
726 AMC Loews 19th St. East 6
NO. 1 CHINESE RESTAURANT
727 No 1 Chinese Restaurant
POTBELLY SANDWICH WORKS
728 Potbelly Sandwich Shop
PIZZA HUT
729 Pizza Hut
ISABELLA'S FROZEN TREATS
730 Isabella's Italian Ices & Ice Cream
BROOKLYN BEANERY COFFEE HOUSE
731 D'Amico Coffee
PASTA WIZ
732 Pasta Wiz
OVEST PIZZOTECA
733 Ovest Pizzoteca by Luzzo's
PIZZA HUT
Yelp hours unavailable!
734 Pizza Hut
LEGEND
735 Legends New York City
OLIVE TREE CAFE & COMEDY CELLAR
736 Olive Tree Cafe
TAPERIA
737 Taperia
PIZZA HUT
Yelp hours unavailable!
738 Pizza Hut
STARBUCKS
Yelp hours unavailable!
739 Starbucks
SQUIRE DINER
740 Squire's Dine

...

1048 Snowdonia
PIZZA PLUS KOSHER PIZZA
1049 Pizza Plus
NORTHERN DUMPLING KING
1050 Northern King Dumpling - Wang Mandu
PHO BAR
1051 PhoBar
ST JAMES GATE
1052 St James Gate
NEW KAM LAI
1053 New Kam Lai
PARKSIDE LOUNGE
1054 Parkside Lounge
NATHAN'S FAMOUS
1055 Nathan's Famous
LA PECORA BIANCA
1056 La Pecora Bianca
SUGAR BAR
1057 Ashford & Simpson's Sugar Bar
SUBWAY
Yelp hours unavailable!
1058 Subway
SHANGHAI SOUP DUMPLING
1059 Shanghai 21
Barnea Bistro
1060 Barnea Bistro
PIZZKEN
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -74.10621643066406, 'latitude': 40.58500584629235}}}
1061 None
CINNABON/STRAWBERRY ICE COLD
Yelp hours unavailable!
1062 Carvel
POPEYES LOUISIANA KITCHEN
1063 Popeyes Louisiana Kitchen
PANERA BREAD
1064 Panera Bread
LAS JUANAS RESTAURANTE ASADERO
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -73.817138671875, 'latitude': 40.764758281851485}}}
1065 None
GREGORYS COFFEE
1066 Gregorys Coffee
OUI OUI CREPES AND PIZZA
1067 Oui Oui C

BRUNO
1213 Bruno Pizza
CITY CAFE
Yelp hours unavailable!
1214 City Cafe
MORGENSTERN'S FINEST ICE CREAM
1215 Morgenstern's Finest Ice Cream
A-JIAO SICHUAN CHINESE
1216 Wa Jeal Sichuan Chili House
METLIFE EXECUTIVE DINING ROOM
1217 Grand Central Terminal
MISTER HOTPOT
1218 Mister Hotpot
MOMOFUKU
1219 Momofuku Nishi
MOE'S SOUTHWEST GRILL
1220 Moe's Southwest Grill
DOLCINO TRATTORIA TOSCANA
1221 Dolcino Trattoria Toscana Italian
DARKSTAR COFFEE + ESPRESSO
1222 Darkstar Coffee + Espresso
DARLING'S
1223 Darlings
MOKBAR
1224 Mokbar Brooklyn
Gohan Ya
1225 Gohan-ya
CROWN FRIED CHICKEN
Yelp hours unavailable!
1226 Crown Fried Chicken
LA ABUNDANCIA
Yelp hours unavailable!
1227 La Abundancia
BLIND PIG
1228 Blind Pig
AROY DEE THAI KITCHEN
1229 Aroy Dee Thai Kitchen
LAMOZA CAFE
1230 Lamoza
CROWN FRIED CHICKEN
Yelp hours unavailable!
1231 Crown Fried Chicken
DOLL'S
1232 FlashDancers Presents: New York Dolls Gentlemen’s Club
ALMOND
1233 Almond
DETOXX BAR
1234 Detoxx Bar
COOKUNITY
1235 CookUnity
67 CIA

1387 Crispy Chicken Restaurant
CUSP
1388 Cusp, Crepe and Espresso Bar
MAISON KAYSER
1389 Maison Kayser
ANCHOR HEIGHTS
1390 Anchor Heights
COOKUNITY
1391 CookUnity
ITALIENNE
1392 Trattoria Italienne
nan
1393 Nan Xiang Xiao Long Bao
MOFONGO EL MOCANO RESTAURANT AND LOUNGE
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -73.90949249267578, 'latitude': 40.84133505538404}}}
1394 None
MOE'S SOUTHWEST GRILL
1395 Moe's Southwest Grill
MIZU SUSHI & DUMONT BURGER
1396 1 or 8
MAMAMOO LOCAL ORGANIC KITCHEN
1397 Mamamoo
IRVING PLAZA
1398 Irving Plaza
LITTLE OCHIE JAMAICAN CUISINE
Yelp hours unavailable!
1399 Little Ochie Jamaican Cuisine
CROWN HEIGHTS BUNCH-O-BAGELS AND MORE
1400 Russ & Daughters
PEKING HOUSE
Yelp hours unavailable!
1401 Peking House
BUNKER
1402 Bunker
CHUNG WAH
1403 Chung Wah Barber Shop
BETTY'S CATERING
Yelp hours unavailable!
1404 Betty's Total Events
DIAMOND KRUST
Yelp hours unavailable!
1405 Diamond Krust Grill
OK CAFE
1406 OK Cafe
CAFFE BENE
1407 Caffe Bene


APPLEBEE'S
1572 Applebee's Grill + Bar
NAP CAFE
1573 Koneko
LA CASA DEL POLLO RESTAURANT
Yelp hours unavailable!
1574 La Casa Del Pollo Peruano
CROWN FRIED CHICKEN
1575 Crown Fried Chicken
BUTTERCUP BAKE SHOP
1576 Buttercup Bake Shop
BUTTER RESTAURANT
1577 Butter Midtown
CONNIE'S PIZZERIA
1578 Connie's Pizzeria
BURGER KING
1579 Burger King
LE PAIN QUOTIDIEN
1580 Le Pain Quotidien
ODDFELLOWS ICE CREAM
1581 OddFellows Ice Cream
CROWN FRIED CHICKEN
Yelp hours unavailable!
1582 Crown Fried Chicken
MAY WAH FAST FOOD
1583 May Wah Fast Food
MAY LUCK KITCHEN
1584 Chase Taste Buffet
CIRO'S PIZZA
Yelp hours unavailable!
1585 Ciro's Pizza
MILSTEIN CAFE
Yelp hours unavailable!
1586 NewYork-Presbyterian/Columbia Univ Medical -  Milstein Cafeteria
MCDONALD'S
1587 McDonald's
MCDONALD'S
1588 McDonald's
ASADERO LA FOGATA
{'businesses': [], 'total': 0, 'region': {'center': {'longitude': -73.86606216430664, 'latitude': 40.74296231280802}}}
1589 None
BURGER KING
1590 Burger King
nan
1591 Yun Nan Flavor Ga

ALENBI KITCHEN
1749 ALENbi
CAFE DUKE
1750 Cafe Duke
NANCY RESTAURANT
Yelp hours unavailable!
1751 Nancy Restaurant
AMURA JAPANESE RESTAURANT
1752 Amura Japanese
16 HANDLES
1753 16 Handles
EL VEZ
1754 El Vez
MISI
1755 Misi
ANAND RESTAURANT
1756 Anand Indian Cuisine
RADEGAST HALL & BIERGARTEN
1757 Radegast Hall and Biergarten
CAFE REGULAR
1758 Café Regular
Dun Huang
1759 Dun Huang
LA CAVA
1760 La Cava
MADISON BISTRO
1761 Madison Bistro
ALIMAMA TEA CAFE
1762 Alimama
MISS LI HENAN CUISINE
1763 Miss Li Henan Cuisine
PARMA RESTAURANT
1764 Parma
1020 BAR
1765 1020 Bar
BAGEL BOB'S
Yelp hours unavailable!
1766 Bagel Bob's
TACOS ALSUADERO
1767 Tacos Al Suadero
ANTOJITOS MEXICANO
Yelp hours unavailable!
1768 Antojito Mexicano
CREPE FACTORY
1769 Crepe Factory
CAFE R
1770 Cafe R
MOMO HIBACHI STEAKHOUSE & BAR
1771 MOMO Hibachi Steak House & Bar
OK LAH!
1772 Ok Lah
nan
1773 Nan Xiang Xiao Long Bao
MOLE CANTINA MEXICANA
1774 Casa Enrique
HOP HOUSE HARLEM
1775 Hop House Harlem
